In [24]:
from nltk.translate.bleu_score import sentence_bleu
import pandas
import pickle
from tqdm import tqdm
import numpy as np

In [50]:
!pip install -U sentence-transformers
!pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 940.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.5 MB/s eta 0:00:00


In [51]:
import gensim.downloader as api
wmd_model = api.load('word2vec-google-news-300')

[=====---------------------------------------------] 10.5% 174.0/1662.8MB downloaded

In [36]:
from sentence_transformers import SentenceTransformer
cos_sim_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
def calculate_metrics(result_df):
  output_bleu_scores = []
  cosine_similarities = []
  wmds = []
  for _, row in tqdm(result_df.iterrows()):
    model_response = row['model_response']
    expected_response = row['expected_response']
    
    # BLEU Similarity
    output_bleu_scores.append(sentence_bleu([expected_response.split()], model_response.split()))

    # Consine Similarity
    model_encoding = cos_sim_model.encode(model_response)
    expected_encoding = cos_sim_model.encode(expected_response)
    consine_similarity = np.dot(model_encoding, expected_encoding)/(np.linalg.norm(model_encoding)*np.linalg.norm(expected_encoding))
    cosine_similarities.append(consine_similarity)
    
    # WMD
    wm_distance = wmd_model.wmdistance(model_response.split(), expected_response.split())
    wmds.append(wm_distance)

  output_bleu_scores = np.array(output_bleu_scores)
  cosine_similarities = np.array(cosine_similarities)
  wmds = np.array(wmds)
  return np.mean(output_bleu_scores), np.mean(cosine_similarities), np.mean(wmds)

In [47]:
# result_paths = ['lora_finetune_r_4', 'lora_finetune_r_8', 'lora_finetune_r_16', 'rosa_finetune_r_16', 'rosa_finetune_r_8', 'rosa_finetune_r_4']
result_paths = ['rosa_finetune_r_4']
BLEU_score = []
cosine_score = []
wmd_scores = []
for result_path in result_paths:
  print("For experiment:", result_path)
  with open(f"./{result_path}.pkl", "rb") as input_file:
    result_df = pickle.load(input_file)
    bleu, cosine, wmd = calculate_metrics(result_df)
    BLEU_score.append(bleu)
    cosine_score.append(cosine)
    wmd_scores.append(wmd)

For experiment: rosa_finetune_r_4


0it [00:00, ?it/s]

/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower orde

In [48]:
print(result_paths)
print(BLEU_score)
print(cosine_score)
print(wmd_scores)

['rosa_finetune_r_4']
[0.02079812071505654]
[0.5342936]
